In [1]:
import numpy as np
from astropy import units as u
from astropy.table import Table, join
from astropy.io import fits
from astropy.nddata import CCDData
from astropy.stats import mad_std, sigma_clipped_stats
import ccdproc
from ccdproc import ImageFileCollection, Combiner, combine
import matplotlib.pyplot as plt
import photutils
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry 
from photutils.background import MMMBackground, MADStdBackgroundRMS
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_sigma_to_fwhm

In [2]:
#Paths updated for each galaxy
#Not the most elegant solution, but I wanted to make sure I was using the right files
bias_path = ImageFileCollection('Data/Bias/')
darks_path = ImageFileCollection('Data/Darks/')
flats_path = ImageFileCollection('Data/M64/')
M64 = ImageFileCollection('Data/M64/')

In [3]:
calibrated_biases = bias_path.files_filtered(imagetyp='bias', include_path=True)
combined_bias = ccdproc.combine(calibrated_biases,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std,
                             mem_limit=350e6, unit=u.adu
                            )

combined_bias.meta['combined'] = True

combined_bias.write('./Data/Reduced/combined_bias.fit', overwrite = False)

In [ ]:
for flat, file_name in flats_path.ccds(imagetyp='flat', return_fname=True, ccd_kwargs=dict(unit='adu')):
    reduced = ccdproc.subtract_bias(flat, combined_bias)
    reduced.write('./Data/M64/Reduced/'+ file_name, overwrite = False)

In [5]:
flat_imagetyp = 'flat'
flats_path = ImageFileCollection('Data/M64')
flat_filters = set(h['filter'] for h in flats_path.headers(imagetyp=flat_imagetyp))

def inv_median(a):
    return 1 / np.median(a)

for filt in flat_filters:
    to_combine = flats_path.files_filtered(imagetyp=flat_imagetyp, filter=filt, include_path=True)
    combined_flat = ccdproc.combine(to_combine,
                                 method='average', scale=inv_median,
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                                 mem_limit=350e6, unit=u.adu
                                )

    combined_flat.meta['combined'] = True
    dark_file_name = 'combined_flat_filter_{}.fit'.format(filt.replace("''", "p"))
    combined_flat.write('./Data/M64/Reduced/' + dark_file_name, overwrite = False)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.


In [6]:
ifc_reduced = ImageFileCollection('Data/M64/Reduced/')
combined_flats = {ccd.header['filter']: ccd for ccd in ifc_reduced.ccds(imagetyp=flat_imagetyp, combined=True)}

images = ImageFileCollection('Data/M64')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.


In [9]:
for light, file_name in images.ccds(imagetyp='light', return_fname=True, ccd_kwargs=dict(unit='adu')):
    reduced = ccdproc.subtract_bias(light, combined_bias)   
    good_flat = combined_flats[reduced.header['filter']]
    reduced = ccdproc.flat_correct(reduced, good_flat)
    reduced.write('./Data/M64/Reduced/' + file_name, overwrite = False)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
